In [1]:
import torch
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm

import sys
from pathlib import Path


project_root = Path.cwd().parent 
sys.path.append(str(project_root))
sys.path.append("../")


from src.utils import (check_cuda, get_dataset, low_density_anomalies,
                   select_model, setup_experiment)
from src.utils import get_dataset, select_model
from hydra import initialize, compose
from src.shap_explainer import ShapExplainer


2025-01-30 11:03:20.458327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-30 11:03:20.472323: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-30 11:03:20.476571: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 11:03:20.487334: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-30 11:03:21.306539: W tensorflow/comp

In [2]:
def load_model_and_dataset_from_path(experiment_path):
    cfg_experiment_path = Path(f"{experiment_path}/experiment_config.yaml")
    model_path = f"{experiment_path}/model.pth"
        
    with initialize(config_path=str(experiment_path), version_base=None):
        cfg = compose(config_name=cfg_experiment_path.name)
    cfg.dataset.dataset_path = "../" + cfg.dataset.dataset_path
    dataset = get_dataset(cfg)
    X = dataset['X_test']
    y = dataset['y_test']
    explanation = dataset['explanation_test']

    # keep only data with label != 0
    X = X[y != 0]
    explanation = explanation[y != 0]
    y = y[y != 0]

    model = select_model(cfg.model, device="cuda:0" if torch.cuda.is_available() else "cpu")
    model.load_model(model_path, X)
    return model, dataset, cfg

In [3]:
def explanation_accuracy(explanation,ground_truth, k="auto"):
    if explanation.shape != ground_truth.shape:
        raise ValueError(
            "The explanation and ground truth must have the same shape."
        )
    if len(explanation.shape) == 1:
        explanation = explanation.reshape(1, -1)
    if len(ground_truth.shape) == 1:
        ground_truth = ground_truth.reshape(1, -1)
    if type(explanation) is torch.Tensor:
        explanation = explanation.cpu().detach().numpy()
    if type(ground_truth) is torch.Tensor:
        ground_truth = ground_truth.cpu().detach().numpy()
    accuracy = []
    for row in range(ground_truth.shape[0]):
        if k == "auto":
            k_ = int(np.sum(ground_truth[row]))
        else:
            k_ = k
        if k_ == 0 or int(np.sum(ground_truth[row])) == 0:
            continue
        sorted_indices = np.argsort(explanation[row])[::-1]
        instance_explanation = np.zeros_like(explanation[row])
        instance_explanation[sorted_indices[:k_]] = 1

        instance_accuracy = (
            np.sum(ground_truth[row] * instance_explanation) / k_
        )
        accuracy.append(instance_accuracy)
    return np.mean(accuracy)


def dcg_score_matrix_p(importance_scores, relevance_matrix, p):
    """
    Compute the DCG scores at a given cutoff rank p.
    """
    importance_scores = np.array(importance_scores)
    relevance_matrix = np.array(relevance_matrix)
    importance_scores = importance_scores.squeeze()
    relevance_matrix = relevance_matrix.squeeze()
    assert (
        importance_scores.shape == relevance_matrix.shape
    ), "importance_scores and relevance_matrix must have the same shape"

    # Sort relevance based on importance scores
    if len(importance_scores.shape) == 1:
        importance_scores = importance_scores.reshape(1, -1)
        relevance_matrix = relevance_matrix.reshape(1, -1)

    sorted_indices = np.argsort(importance_scores, axis=1)[:, ::-1]
    sorted_relevance = np.take_along_axis(
        relevance_matrix, sorted_indices, axis=1
    )

    # Consider only the top p items
    sorted_relevance_p = sorted_relevance[:, :p]
    ranks = np.arange(1, p + 1)  # Ranks from 1 to p

    # Compute DCG scores
    dcg_scores = np.sum(sorted_relevance_p / np.log2(ranks + 1), axis=1)

    return dcg_scores


def idcg_score_matrix_p(relevance_matrix, p):
    """
    Compute the IDCG scores at a given cutoff rank p.
    """
    if len(relevance_matrix.shape) == 1:
        relevance_matrix = relevance_matrix.reshape(1, -1)
    relevance_matrix = np.array(relevance_matrix)
    sorted_relevance = np.sort(relevance_matrix, axis=1)[:, ::-1]

    # Consider only the top p items
    sorted_relevance_p = sorted_relevance[:, :p]
    ranks = np.arange(1, p + 1)  # Ranks from 1 to p

    # Compute IDCG scores
    idcg_scores = np.sum(sorted_relevance_p / np.log2(ranks + 1), axis=1)

    return idcg_scores


def nDCG_(importance_scores, relevance_matrix, p):
    """
    Compute the nDCG scores at a given cutoff rank p.
    """
    dcg_scores_p = dcg_score_matrix_p(importance_scores, relevance_matrix, p)
    idcg_scores_p = idcg_score_matrix_p(relevance_matrix, p)

    # Compute normalized DCG
    ndcg_scores_p = np.zeros_like(dcg_scores_p)
    for i in range(len(dcg_scores_p)):
        if idcg_scores_p[i] == 0:
            ndcg_scores_p[i] = 0
        else:
            ndcg_scores_p[i] = dcg_scores_p[i] / idcg_scores_p[i]

    return ndcg_scores_p


def nDCG_p(importance_scores, relevance_matrix, k= 'auto'):
    nDCG_scores = []
    if len(importance_scores.shape) == 1:
        importance_scores = importance_scores.reshape(1, -1)
    if len(relevance_matrix.shape) == 1:
        relevance_matrix = relevance_matrix.reshape(1, -1)
    for i in range(importance_scores.shape[0]):
        if k == "auto":
            k_ = int(np.sum(relevance_matrix[i]))
        else:
            k_ = k
        if k_ == 0 or int(np.sum(relevance_matrix[i])) == 0:
            continue
        nDCG_scores.append(nDCG_(importance_scores[i], relevance_matrix[i], p=k_))
    return np.array(nDCG_scores)


In [4]:
experiment_path = "../results/all_db_all_training/DTEC_DSIL_deterministic_0.5_s0_T400_bins7/A_synthetic_f1000_s5000_c40_r0.01_0.01_0.01_0.01_0.01_seed_1"

In [5]:
import time


def run_explanation(model, data, saving_path, cfg, diffusion_perturbation_step=50, shap_ratio=1.5):
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    score = model.predict_score(data["X_test"], device=device).squeeze()
    data["y_test"][data["y_test"] > 0] = 1
    metric_df = pd.read_csv(Path(saving_path, "model_metrics.csv"))
    indices = np.arange(len(data["y_test"]))
    # Suppose we know how much anomaly are in the dataset
    y_pred = low_density_anomalies(-score, len(indices[data["y_test"] == 1]))
    pred_indices = np.arange(len(data["X_test"]))[y_pred == 1]
    samples = data["X_test"][pred_indices]
    expected_explanation = data["explanation_test"][pred_indices]
    truth_label = data["y_test"][pred_indices]
    # Keep only the samples for which turth_lab is not 0
    samples = samples[truth_label != 0]
    expected_explanation = expected_explanation[truth_label != 0]
    truth_label = truth_label[truth_label != 0]
    existing_columns = metric_df.columns
    force_rerun = True
    print(f'Number of samples: {len(samples)}')
    cfg.output_path = "../" + cfg.output_path
    saving_path, experiment_name = setup_experiment(cfg)
    if "new_mean_diffusion_accuracy" not in existing_columns or force_rerun:
        print("Mean diffusion...")
        start = time.time()
        explanation = model.instance_explanation(
            samples, saving_path=saving_path, step=diffusion_perturbation_step, agg="mean"
        )
        end = time.time()
        nDCG = nDCG_p(explanation, expected_explanation).mean()
        accuracy = explanation_accuracy(explanation, expected_explanation).mean()
        mean_diffusion_explanation_time = end - start
        metric_df["new_mean_diffusion_accuracy"] = accuracy
        metric_df["new_mean_diffusion_ndcg"] = nDCG
        metric_df["new_mean_diffusion_time"] = mean_diffusion_explanation_time
        metric_df["mean_new_mean_diffusion_time"] = mean_diffusion_explanation_time / len(samples)
        np.save(Path(saving_path, f"new_mean_diffusion_explanation.npy"), explanation)
        print(f'nDCG: {nDCG}, accuracy: {accuracy}')
        print(f'Mean computation time: {mean_diffusion_explanation_time / len(samples)}')
    if "new_max_diffusion_accuracy" not in existing_columns or force_rerun:
        print("Max diffusion...")
        start = time.time()
        explanation = model.instance_explanation(
            samples, saving_path=saving_path, step=diffusion_perturbation_step, agg="max"
        )
        end = time.time()
        nDCG = nDCG_p(explanation, expected_explanation).mean()
        accuracy = explanation_accuracy(explanation, expected_explanation).mean()
        max_diffusion_explanation_time = end - start
        metric_df["new_max_diffusion_accuracy"] = nDCG
        metric_df["new_max_diffusion_ndcg"] = accuracy
        metric_df["new_max_diffusion_time"] = max_diffusion_explanation_time
        metric_df["mean_new_max_diffusion_time"] = max_diffusion_explanation_time / len(samples)
        np.save(Path(saving_path, f"new_max_diffusion_explanation.npy"), explanation)
        print(f'nDCG: {nDCG}, accuracy: {accuracy}')
        print(f'Mean computation time: {max_diffusion_explanation_time / len(samples)}')

    if "adatative_shap_explanation_accuracy" not in existing_columns or force_rerun:
        print('SHAP...')
        subset = np.random.choice(
            np.arange(len(data["X_train"])), size=int(0.02 * data['X_train'].shape[0]), replace=False
        )
        subset = data["X_train"][subset]
        n_samples = int(shap_ratio * data["X_test"].shape[1])
        print()
        start = time.time()
        shap_explainer = ShapExplainer(model, subset)
        explanation = shap_explainer.explain_instance(
            samples,
            nsamples=n_samples,
        ).squeeze()
        end = time.time()
        shap_nDCG = nDCG_p(explanation, expected_explanation).mean()
        shap_accuracy = explanation_accuracy(explanation, expected_explanation).mean()
        shap_explanation_time = end - start
        metric_df['new_adaptative_shap_accuracy'] = shap_accuracy
        metric_df['new_adaptative_shap_ndcg'] = shap_nDCG
        metric_df['new_adaptative_shap_time'] = shap_explanation_time
        metric_df['new_mean_adaptative_shap_time'] = shap_explanation_time / len(samples)
        np.save(Path(saving_path, f"new_shap_diffusion_explanation.npy"), explanation)
        print(f'nDCG: {shap_nDCG}, accuracy: {shap_accuracy}')
        print(f'Mean computation time: {shap_explanation_time / len(samples)}')
    # Save metrics df
    metric_df.to_csv(Path(saving_path, "model_metrics.csv"))
    np.save(Path(saving_path, f"expected_explanation.npy"), expected_explanation)
    print(f'Sved at {saving_path}')

In [6]:
def process_each_dataset_version(experiment_path, seeds=[], diffusion_perturbation_step=50, shap_r=1.5):
    for seed in seeds:
        print(f"Processing {experiment_path} with seed {seed}")
        seed_experiment_path = f"{experiment_path}_seed_{seed}"
        model, dataset, cfg = load_model_and_dataset_from_path(seed_experiment_path)
        run_explanation(model, dataset, seed_experiment_path, cfg, diffusion_perturbation_step, shap_r)


In [11]:
training_mode = ["DSIL_deterministic_0.5"]
datasets_name = [
    # "A_synthetic_f4_s5000_c2_r0.05_0.05",
    # "A_synthetic_f4_s5000_c2_r0.025_0.025",
    # "A_synthetic_f10_s5000_c4_r0.033_0.033_0.033",
    # "A_synthetic_f10_s5000_c4_r0.0166_0.0166_0.0166",
    # "A_synthetic_f50_s5000_c12_r0.01_0.01_0.01_0.01_0.01",
    # "A_synthetic_f50_s5000_c12_r0.02_0.02_0.02_0.02_0.02",
    # "A_synthetic_f100_s5000_c40_r0.01_0.01_0.01_0.01_0.01",
    # "A_synthetic_f100_s5000_c40_r0.02_0.02_0.02_0.02_0.02",
    "A_synthetic_f1000_s5000_c40_r0.01_0.01_0.01_0.01_0.01",
    "A_synthetic_f1000_s5000_c40_r0.02_0.02_0.02_0.02_0.02",
]
diffusion_perturbation_per_dataset = {
}

In [ ]:
for training in training_mode:
    for dataset_name in datasets_name:
        experiment_path = f"../results/all_db_all_training/DTEC_{training}_s0_T400_bins7/{dataset_name}"
        process_each_dataset_version(experiment_path, seeds=[0, 1, 2, 3, 4], diffusion_perturbation_step=2, shap_r=1.5)

Processing ../results/all_db_all_training/DTEC_DSIL_deterministic_0.5_s0_T400_bins7/A_synthetic_f1000_s5000_c40_r0.01_0.01_0.01_0.01_0.01 with seed 0
{'Samples': 5000, 'Features': 1000, 'Anomalies': 250, 'Anomalies Ratio(%)': 5.0}


/home/elouan/code/code_repo/src/models/dte.py:418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(path))


Number of samples: 186
Mean diffusion...
nDCG: 0.4846551773477437, accuracy: 0.4697471611941568
Mean computation time: 1.8547912746347406
Max diffusion...
nDCG: 0.4770100992133294, accuracy: 0.46452245032166123
Mean computation time: 1.8467941284179688
SHAP...

nDCG: 0.5114092347911975, accuracy: 0.4759612434661347
Mean computation time: 4.190211961346288
Sved at ../results/all_db_all_training/DTEC_DSIL_deterministic_0.5_s0_T400_bins7/A_synthetic_f1000_s5000_c40_r0.01_0.01_0.01_0.01_0.01_seed_0
Processing ../results/all_db_all_training/DTEC_DSIL_deterministic_0.5_s0_T400_bins7/A_synthetic_f1000_s5000_c40_r0.01_0.01_0.01_0.01_0.01 with seed 1
{'Samples': 5000, 'Features': 1000, 'Anomalies': 250, 'Anomalies Ratio(%)': 5.0}


/home/elouan/code/code_repo/src/models/dte.py:418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(path))


Number of samples: 189
Mean diffusion...
nDCG: 0.4800549645204762, accuracy: 0.4673621012253384
Mean computation time: 1.8194398198808943
Max diffusion...
nDCG: 0.4678547327230751, accuracy: 0.46007358857697195
Mean computation time: 1.8353728882219427
SHAP...

nDCG: 0.5055739067810405, accuracy: 0.47012352308865474
Mean computation time: 4.094736251881514
Sved at ../results/all_db_all_training/DTEC_DSIL_deterministic_0.5_s0_T400_bins7/A_synthetic_f1000_s5000_c40_r0.01_0.01_0.01_0.01_0.01_seed_1
Processing ../results/all_db_all_training/DTEC_DSIL_deterministic_0.5_s0_T400_bins7/A_synthetic_f1000_s5000_c40_r0.01_0.01_0.01_0.01_0.01 with seed 2
{'Samples': 5000, 'Features': 1000, 'Anomalies': 250, 'Anomalies Ratio(%)': 5.0}


/home/elouan/code/code_repo/src/models/dte.py:418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(path))


Number of samples: 181
Mean diffusion...
nDCG: 0.4929435686631867, accuracy: 0.47924660990521994
Mean computation time: 1.897063618865461
Max diffusion...
nDCG: 0.4846662702521386, accuracy: 0.473460637333121
Mean computation time: 1.9432620541166865
SHAP...

